# Example on how to download a specific variable from the Marine Data Store using the Copernicus-Marine Client

## First import the client from pypi

In [ ]:
import copernicus_marine_client as cmc
from datetime import datetime
import xarray as xr
import cdsapi
import os

## How to parameterize the clients credentials

### This turorial is designed to fetch the credentials from your personal vault on the datalab, in a file called "Credentials" at the root of your vault. 
#### List of credentials expected: 
- COPERNICUS_MARINE_SERVICE_USERNAME
- COPERNICUS_MARINE_SERVICE_PASSWORD
- CDS_UID
- CDS_APIKEY


Otherwise, you can also:
1. for the copernicus-marine-client input the credentials at execution when prompted to or use the _login_ command: cmc.login()
2. export the environment variables for the copernicus-marine-client:

In [ ]:
#os.environ['COPERNICUS_MARINE_SERVICE_USERNAME'] = {username}
#os.environ['COPERNICUS_MARINE_SERVICE_PASSWORD'] = {password}

#### To create the .cdsapirc file manually:

In [ ]:
#with open('../.cdsapirc', 'w') as f:
#    f.write("url: https://cds.climate.copernicus.eu/api/v2\nkey: {CDS_UID}:{CDS_APIKEY}")

## How to find the right 'dataset_id' ?

### Based on a set of keywords find the most appropriate product

In [ ]:
query_keywords = ["Global Ocean Physics Reanalysis"]

In [ ]:
query_result = cmc.describe(contains=query_keywords)
query_result

### Let's assume we want the product 'Global Ocean Physics Reanalysis'

### Next fetch the info on datasets for this product

In [ ]:
query_result = cmc.describe(contains=['Global Ocean Physics Reanalysis'], include_datasets=True)
for product in query_result.get('products'):
    for dataset in product.get('datasets'):
        print(f"{dataset.get('dataset_id')} - {dataset.get('dataset_name')}")

### Assuming one wants the dataset on monthly mean reanalysis potential temperature: dataset_id = 'cmems_mod_glo_phy_my_0.083_P1M-m'

In [ ]:
for product in query_result.get('products'):
    for dataset in product.get('datasets'):
        if dataset.get('dataset_id') == "cmems_mod_glo_phy_my_0.083_P1M-m":
            chosen_dataset = dataset
print("---Variables---")
for variable in chosen_dataset.get("variables"):
    print(f"{variable.get('standard_name')} - {variable.get('short_name')}")

## Fetch a subset of this dataset

In [ ]:
start_datetime=datetime(2020,1,12)
end_datetime=datetime(2020,2,12)

test_data=cmc.load_xarray_dataset(
    dataset_id="cmems_mod_glo_phy_my_0.083_P1M-m", 
    variables=["so"],
    minimal_longitude=44.8,
    maximal_longitude=45.6,
    minimal_latitude=-4.6,
    maximal_latitude=-3.7,
    minimal_depth=0.0,
    maximal_depth=2.0,
    start_datetime=start_datetime,
    end_datetime=end_datetime,
)

In [ ]:
test_data

In [ ]:
test_data.so.isel(time=0, elevation=0).plot()

## Fetch from Climate Data Store

In [ ]:
c = cdsapi.Client()
c.retrieve('reanalysis-era5-complete', { # Requests follow MARS syntax
                                        # Keywords 'expver' and 'class' can be dropped. They are obsolete
                                        # since their values are imposed by 'reanalysis-era5-complete'
   'date'    : '2013-01-01',            # The hyphens can be omitted
   'levelist': '1/10/100/137',          # 1 is top level, 137 the lowest model level in ERA5. Use '/' to separate values.
   'levtype' : 'ml',
   'param'   : '130',                   # Full information at https://apps.ecmwf.int/codes/grib/param-db/
                                        # The native representation for temperature is spherical harmonics
   'stream'  : 'oper',                  # Denotes ERA5. Ensemble members are selected by 'enda'
   'time'    : '00/to/23/by/6',         # You can drop :00:00 and use MARS short-hand notation, instead of '00/06/12/18'
   'type'    : 'an',
   'area'    : '80/-50/-25/0',          # North, West, South, East. Default: global
   'grid'    : '1.0/1.0',               # Latitude/longitude. Default: spherical harmonics or reduced Gaussian grid
   'format'  : 'netcdf',                # Output needs to be regular lat-lon, so only works in combination with 'grid'!
}, 'ERA5-ml-temperature-subarea.nc')     # Output file. Adapt as you wish.

In [ ]:
era5=xr.open_dataset("ERA5-ml-temperature-subarea.nc")

In [ ]:
era5.isel(level=1, time=0).t.plot()